In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/IW06-07

In [ ]:
!pip3 install deepspeech-gpu
!pip3 install jiwer

In [ ]:
import deepspeech
import wave
import numpy as np
import torch
import torchaudio
from jiwer import compute_measures
import pickle

In [ ]:
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.pbmm
!curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.9.3/deepspeech-0.9.3-models.scorer

In [ ]:
model_file_path = 'deepspeech-0.9.3-models.pbmm'
model = deepspeech.Model(model_file_path)
scorer_file_path = 'deepspeech-0.9.3-models.scorer'
model.enableExternalScorer(scorer_file_path)
lm_alpha = 0.931289039105002
lm_beta = 1.1834137581510284
model.setScorerAlphaBeta(lm_alpha, lm_beta)
def convert(audio):
  w = wave.open(audio, 'r')
  assert int(w.getframerate()) == 16000
  data = np.frombuffer(w.readframes(w.getnframes()), dtype=np.int16)
  w.close()
  return model.stt(data)

In [ ]:
enh_path = './deepxi/DeepXi/out/mhanet-1.1c/mmse-lsa/'
adv_path = './deepxi/DeepXi/Adversarial_Examples'
grounds = ['/long-ground.txt', '/medium-ground.txt', '/short-ground.txt']
files = ['/long-signals.txt', '/medium-signals.txt', '/short-signals.txt']

In [ ]:
trans_dict = {}
translations = []
for f, g in zip(files, grounds):
  t = open(adv_path+f, 'r')
  text = t.read().strip().split('\n')
  gr = open(adv_path+g, 'r')
  ground = gr.read().strip().split('\n')
  trans = {}
  for i in range(2, 102):
    temp = {}
    temp['base'] = ground[i-2].split('\t')[1]
    ori_trans = convert(adv_path + '/' + text[i])
    enh_ori_trans = convert(enh_path + '/' + text[i])
    # long, medium, short adv
    lms = (text[i+101], text[i+2*101], text[i+3*101])
    adv_trans = [convert(adv_path+'/'+name) for name in lms]
    enh_trans = [convert(enh_path+'/'+name) for name in lms]
    temp['ori'] = ori_trans
    temp['adv'] = adv_trans
    temp['enh'] = enh_trans
    temp['enh_ori'] = enh_ori_trans
    trans[text[i]] = temp
  translations.append(trans)
  t.close()
  gr.close()

trans_dict['long'] = translations[0]
trans_dict['medium'] = translations[1]
trans_dict['short'] = translations[2]

In [ ]:
trans_dict['long'] = translations[0]
trans_dict['medium'] = translations[1]
trans_dict['short'] = translations[2]

In [ ]:
with open('deepxi_trans.pickle', 'wb') as e:
  pickle.dump(trans_dict, e, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle

In [ ]:
wer = {}
with open('deepxi_trans.pickle', 'rb') as e:
  wer = pickle.load(e)

In [ ]:
wer.keys()

In [ ]:
wer['short']['sample-006936.wav']